This notebook is for calculating PIQE and BRISQUE of image folder

In [ ]:
!pip -q install pyiqa
!pip -q install pypiqe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.0/248.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 25.2 MB/s e

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/
# !pip install kaggle
!mkdir -v ~/.kaggle

!cp -f "/content/drive/MyDrive/Thesis Resource/kaggle.json" ~/.kaggle
!kaggle datasets download -d quandang/nomnaocr
!kaggle datasets download -d ngcthunhb/nomna-sr-patches
!kaggle datasets download -d ngcthunhb/nomdataset
!kaggle datasets download -d ngcthunhb/nomdataset-crops

/content
mkdir: created directory '/root/.kaggle'
Dataset URL: https://www.kaggle.com/datasets/quandang/nomnaocr
License(s): CC-BY-NC-SA-4.0
 99% 416M/421M [00:03<00:00, 119MB/s]
100% 421M/421M [00:03<00:00, 131MB/s]
Dataset URL: https://www.kaggle.com/datasets/ngcthunhb/nomna-sr-patches
License(s): unknown
100% 9.15G/9.16G [01:28<00:00, 217MB/s]
100% 9.16G/9.16G [01:28<00:00, 111MB/s]
Dataset URL: https://www.kaggle.com/datasets/ngcthunhb/nomdataset
License(s): unknown
 87% 67.0M/77.0M [00:00<00:00, 92.0MB/s]
100% 77.0M/77.0M [00:00<00:00, 99.6MB/s]
Dataset URL: https://www.kaggle.com/datasets/ngcthunhb/nomdataset-crops
License(s): unknown
 88% 160M/182M [00:01<00:00, 104MB/s]
100% 182M/182M [00:01<00:00, 101MB/s]


In [ ]:
!unzip -q /content/nomnaocr.zip -d /content/Datasets
!unzip -q /content/nomna-sr-patches.zip -d /content/datasets
!unzip -q /content/nomdataset.zip -d /content/Nomdatasets
!unzip -q /content/nomdataset-crops.zip -d /content/NomDatasetCrop

In [ ]:
import os
from tqdm.auto import tqdm
import glob
import cv2
import matplotlib.pyplot as plt


def count_images(directory):
  """
  Counts the number of images in a directory and its subdirectories.

  Args:
    directory: The directory to count images in.

  Returns:
    The total number of images found.
  """

  image_count = 0
  for root, _, files in os.walk(directory):
    for file in files:
      if file.lower().endswith((".jpg", ".png", ".jpeg", ".gif")):
        image_count += 1
  return image_count

IMG_DIR = '/content/datasets/RealESGRANx2plus_RealCE_1K'

# Example usage
image_count = count_images(IMG_DIR)
print(f"Total number of images: {image_count}")


Total number of images: 38118


In [ ]:
import pyiqa
import torch
from multiprocessing import Pool, set_start_method
from tqdm.auto import tqdm
import glob
import os
import cv2

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

iqa_metric = pyiqa.create_metric('brisque', device=device)

print('Lower is better: ', iqa_metric.lower_better)

def worker(img_path):
    score_fr = iqa_metric(img_path)
    return score_fr

avg_score = 0

for i in os.listdir(IMG_DIR):
#   Check i is folder
    if os.path.isdir(os.path.join(IMG_DIR, i)):
        input_paths = glob.glob(os.path.join(IMG_DIR, i, '*'))
        for j in tqdm(input_paths):
            if not j.endswith((".jpg", ".png", ".jpeg", ".gif")):
                continue
            # Check image shape
            img = cv2.imread(j)
            # print(img.shape)
            if img.shape <= (12, 12, 3):
                continue
            score_fr = iqa_metric(j)
            avg_score += score_fr
avg_score /= image_count
print('Avg BRISQUE score: ', avg_score)

Downloading: "https://github.com/chaofengc/IQA-PyTorch/releases/download/v0.1-weights/brisque_svm_weights.pth" to /root/.cache/torch/hub/checkpoints/brisque_svm_weights.pth



100%|██████████| 112k/112k [00:00<00:00, 8.94MB/s]


Lower is better:  True


  0%|          | 0/38118 [00:00<?, ?it/s]

Avg BRISQUE score:  tensor([32.0207], device='cuda:0')


In [ ]:
from pypiqe import piqe
from tqdm.auto import tqdm
import glob
import os
import cv2

avg_score = 0

for i in os.listdir(IMG_DIR):
#   Check i is folder
    if os.path.isdir(os.path.join(IMG_DIR, i)):
        input_paths = glob.glob(os.path.join(IMG_DIR, i, '*'))
        for j in tqdm(input_paths):
            if not j.endswith((".jpg", ".png", ".jpeg", ".gif")):
                continue
            img = cv2.imread(j)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            score, activityMask, noticeableArtifactMask, noiseMask = piqe(img)
            avg_score += score
avg_score /= image_count
print('Avg PIQE score: ', avg_score)

  0%|          | 0/38118 [00:00<?, ?it/s]

Avg PIQE score:  41.21512626219338
